<a href="https://colab.research.google.com/github/jgak12/WormTrackV2/blob/main/WormTrackerV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

WORM TRACKER

TO DO:

* pull small format images from large images based on difference tracked location (done)
* Find a way to relabel all images in the img file so they can be marked which have worms and which dont (kindof done?)
* If time, write quick file relabeling program that gives all files numbered sequence (done
* Write function that turns video into series of .jpg with ordered filenames
* Make user-interface to select intitial worm positions/#worms
* UI to select desired outputs
* UI to select type of experiment


In [ ]:
#On startup in vscode:
#Mac: source /path/to/venv/bin/activate
#Windows: \pathtovenv\Scripts\activate

In [4]:
# %pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# # Authenticate and create the PyDrive client.
# # This only needs to be done once per notebook. Use your UPR account 
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [5]:
def loadFromDrive(file_id, local_name):
  downloaded = drive.CreateFile({'id': file_id})
  downloaded.GetContentFile(local_name)

Read Data


Forget that noise, I'm trying from scratch


In [8]:

###install protobuf somehow


#import tensorflow as tf

# %pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# # Authenticate and create the PyDrive client.
# # This only needs to be done once per notebook. Use your UPR account 
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

#%pip install pycocotools
#%git clone https://github.com/tensorflow/models

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# New Section

In [11]:
#Probs wont need this at all
#%pip install labelImg

#%pip install -U scikit-image
#%pip install -U scikit-learn
#%pip install pandas
#%pip install keras
#%pip install tensorflow



     -------------------------------------- 438.0/438.0 MB 2.1 MB/s eta 0:00:00
     ------------------------------------- 462.5/462.5 KB 28.3 MB/s eta 0:00:00
     -------------------------------------- 126.7/126.7 KB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 42.6/42.6 KB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 KB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 30.7 MB/s eta 0:00:00
     --------------------------------------- 14.2/14.2 MB 22.6 MB/s eta 0:00:00
     ---------------------------------------- 5.8/5.8 MB 28.3 MB/s eta 0:00:00
  Using cached protobuf-3.20.1-cp310-cp310-win_amd64.whl (903 kB)
     ---------------------------------------- 3.4/3.4 MB 24.5 MB/s eta 0:00:00
     ---------------------------------------- 2.8/2.8 MB 25.3 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 KB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing meta

In [1]:
#Import everything

import glob, os 
import skimage
from skimage import io, transform
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.models import Sequential ##Might change this to Convolutional
from keras.layers import Activation, Dropout, Flatten, Dense
#from keras.optimizers import SGD, Adam
from keras.utils.np_utils import to_categorical
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Convolution3D
from keras.layers.convolutional import Conv2D
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix, classification_report
plt.rcParams['axes.grid']=False
import matplotlib as mpl
mpl.rcParams['image.cmap'] = 'viridis'

import matplotlib.pyplot as plt
import numpy as np


In [2]:


img1= plt.imread('/content/drive/MyDrive/WormTracker/WormImgs/wormImgLowQ_181.jpg')
img2= plt.imread('/content/drive/MyDrive/WormTracker/WormImgs/wormImgLowQ_182.jpg')

def frame_compare(img1,img2,showImg=False):
  #plt.imshow(img1)
  #plt.imshow(img2)
  #sum=img2-img1
  sum= np.zeros(img2.shape)

  rowLen=sum.shape[0]
  colLen=sum.shape[1]

  for pixelx in range(0,colLen):
    for pixely in range(0,rowLen):
      for pixelrgb in range(0,3):
        if img2[pixely,pixelx,pixelrgb] >= img1[pixely,pixelx,pixelrgb]:
          sum[pixely,pixelx,pixelrgb]= img2[pixely,pixelx,pixelrgb]-img1[pixely,pixelx,pixelrgb]
        else:
          sum[pixely,pixelx,pixelrgb]= img1[pixely,pixelx,pixelrgb]-img2[pixely,pixelx,pixelrgb]

  ##print(img1[1:5,1:5])
  #print(img2[1:5,1:5])
  #print(sum[1:5,1:5])

  #print(sum.shape[0])


  dif_sens=40 #remove later
  dif_sense= 10

  motion_sensitivity=1.1 * 10

  xavg=0
  yavg=0


  #for xx in range(0, sum.shape[1]):
  # for yy in range(0, sum.shape[0]):
  #   for zz in range(0, sum.shape[2]):
  #     if sum[yy, xx, zz]<= dif_sens:
  #        sum[yy,xx,zz]=0


  #print(sum[0:10, 0:10])

  difArray= np.zeros((sum.shape[0],sum.shape[1]))
  #print(difArray.shape)
  totPixelDif=0;
  numNonZpixels=0
  for pixelx in range(0,colLen):
    for pixely in range(0,rowLen):
      for pixelrgb in range(0,3):
        chanPixelDif= sum[pixely,pixelx,pixelrgb]
        totPixelDif = totPixelDif + chanPixelDif
      if totPixelDif > dif_sense*3:
        #print('hi')
        difArray[pixely,pixelx]=totPixelDif
      totPixelDif=0

  #print(difArray)
  difArray[:,:]= difArray[:,:]/3
  #print(difArray)

  #print(difArray)
  weightTot=0
  for pixelx in range(0,colLen):
    for pixely in range(0,rowLen):
      xavg=xavg + (difArray[pixely,pixelx]*pixelx)
      yavg=yavg + (difArray[pixely,pixelx]*pixely)

      weightTot= weightTot + difArray[pixely,pixelx]

  if xavg>motion_sensitivity:
    xavg=round(xavg/weightTot)
    yavg=round(yavg/weightTot)
    pos=[xavg, yavg]
  else: #returns 00 if change below threshold
    xavg=1
    yavg=1
    pos=[xavg, yavg]

  size=3

  #imshow
  #print(difArray)
  if showImg==True:
    plt.subplot(2,2,1)
    plt.imshow(sum)
    plt.subplot(2,2,2)
    plt.imshow(difArray)
    plt.subplot(2,2,3)
    difArrayMarked= difArray
    #difArrayMarked[round(yavg-size):round(yavg+size), round(xavg-size):round(xavg+size)]= 1000000
    plt.imshow(difArrayMarked)
    print(xavg,yavg)
    plt.scatter(xavg,yavg)
    plt.subplot(2,2,4)
    plt.imshow(img2)

  plt.subplot(1,1,1)
  plt.imshow(img2)
  plt.scatter(xavg,yavg)

  return pos


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/WormTracker/WormImgs/wormImgLowQ_181.jpg'

In [ ]:
a=np.array([5,5,5])
b=np.array([6,2,2])
c=a-b
print(c)
print(img1.shape)

[-1  3  3]
(400, 512, 3)


In [ ]:
def dataset(file_list, flattened=False): #,size=(300,180)
	data = []
	for i, file in enumerate(file_list):
		image = io.imread(file)
		#image = transform.resize(image, size, mode='constant')
		if flattened:
			image = image.flatten()

		data.append(image)

	#labels = [1 if f.split("/")[-1][0] == 'P' else 0 for f in file_list]

	return np.array(data)#, np.array(labels)


In [3]:
#loading list of images
path="/content/drive/MyDrive/WormTracker/WormImgs/" #path to image folder
imlist = glob.glob(os.path.join(path, '*.jpg')) #pulls list of all files in folder
print('Dataset contains {} images'.format(len(imlist))) #returns how many images in list
listlen= len(imlist)

print(imlist) #For some reason, glob pulls the files in a wrong order
imlist.sort() #this sorts them alphabetically ASSUMES IMAGES ARE NUMBERED ALREADY/PROPERLY
print(imlist)

# Load the data. The same as pollen dataset read-skimage (May take a few seconds)
#data,labels = dataset(imlist) 
data= dataset(imlist)
data= np.array(data)
#print(data)
#this loads the images as a numpy ndarray with each image represented as matrix of pixel values

NameError: ignored

In [2]:
#print(img1)
print(data.shape)

NameError: ignored

In [ ]:
stepSize=25
#poslist= np.zeros((listlen-1, 2)) #x,y
poslist= np.empty((0,2), int)


for i in range(0,listlen-1,stepSize):
  img1=data[i,:,:,:]
  img2=data[i+1,:,:,:]

  #frame_compare
  pos=frame_compare(img1,img2)
  print(pos)
  
  poslist= np.append(poslist,np.array([pos]),axis=0)
  #All i want to do is add new paired x and y values to a new row... how is this
  #so much to ask



print(poslist)


In [ ]:
plt.scatter(poslist[:,0],poslist[:,1] )
plt.gca().invert_yaxis()
plt.xlim([0, 512])
plt.ylim([0,400])

In [ ]:
def get_surround_image(img, xpos, ypos):
  xlen=img.shape(0)






In [ ]:
def pullSub(inImg, pos, outputSize):
  if outputSize[0]>inImg.shape[0] or outputSize[1]>inImg.shape[1]:
    return "output bigger than image"
  xmin=int(pos[0]-(outputSize[0]/2))
  xmax=int(pos[0]+(outputSize[0]/2))
  ymin=int(pos[1]-(outputSize[1]/2))
  ymax=int(pos[1]+(outputSize[1]/2))
  if xmin<0:
    xmin=0
  if xmax>(inImg.shape[1]):
    xmax=(inImg.shape[1]-1)
  if ymin<0:
    ymin=0
  if ymax>(inImg.shape[0]):
    ymax=(inImg.shape[0]-1)

  subImg=inImg[ymin:ymax, xmin:xmax]

  return subImg




In [ ]:
img1= plt.imread('/content/drive/MyDrive/WormTracker/WormImgs/wormImgLowQ_187.jpg')
img2= plt.imread('/content/drive/MyDrive/WormTracker/WormImgs/wormImgLowQ_188.jpg')
pos1=frame_compare(img1,img2)
plt.imshow(img1)


In [ ]:
a= pullSub(img2, pos1, [64,64])
plt.imshow(a)

In [ ]:

data = '/content/drive/My Drive/WormTracker/WormImgs/'
#ldseg=np.array(os.listdir(data))
i = 1


#loading list of images
path="/content/drive/MyDrive/WormTracker/WormImgs/" #path to image folder

def numToIndex(n,numDigits):
  out=''
  num=str(n)
  for i in range(0,numDigits-len(num)):
    out= out+'0'
  out= out+ num
  return out

def sort_rename(path,):
  #Takes in a path to a file of numbered, sequential images
  #renames all the files in the directory with '_*' sequential number from 1:n added

  ldseg = glob.glob(os.path.join(path, '*.jpg')) #pulls list of all files in folder
  print('Dataset contains {} images'.format(len(ldseg))) #returns how many images in list
  listlen= len(ldseg)

  print(ldseg) #For some reason, glob pulls the files in a wrong order
  ldseg.sort() #this sorts them alphabetically ASSUMES IMAGES ARE NUMBERED ALREADY/PROPERLY
  print(ldseg)

  i=1
  for filename in ldseg:
    dst =  filename.split('_')[0] + '_' + numToIndex(i,4) + '.jpg' #removes original index marker after a "_"
    print(dst)
    print(filename)
    src =filename
    print(src)
    print(dst)
    os.rename(src, dst)
    i += 1

sort_rename(path)

In [ ]:
#JUST A LITTLE TEST BLOCK
ldseg = glob.glob(os.path.join(path, '*.jpg')) #pulls list of all files in folder
print('Dataset contains {} images'.format(len(ldseg))) #returns how many images in list
listlen= len(ldseg)

print(ldseg) #For some reason, glob pulls the files in a wrong order
ldseg.sort() #this sorts them alphabetically ASSUMES IMAGES ARE NUMBERED ALREADY/PROPERLY
print(ldseg)

NOW THE NEURAL NETWORK:


In [ ]:
# Create a training and testing dataset with 25% of the samples
X_train, X_test, y_train, y_test = train_test_split(data,labels, test_size=.25,random_state=0,)
#ids=np.array(range(data.shape[0]))
#X_train, X_ids, y_train, y_ids = train_test_split(data,ids, test_size=.25,random_state=0,)
#X_test=labels[X_ids]; y_test=labels[y_ids]

In [ ]:
# check shape of the input image to fit with the network 
print(X_train.shape)
print(X_test.shape)

In [ ]:
import skimage
X_train_resized = np.asarray([skimage.transform.resize(image, (224,224)) for image in X_train])
X_test_resized = np.asarray([skimage.transform.resize(image, (224,224)) for image in X_test])

In [1]:
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

y_train_encoded.shape

NameError: ignored

In [ ]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import time
import os


train_gen=ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input,
                horizontal_flip=True,
                samplewise_center=True,
                width_shift_range=.2,
                height_shift_range=.2,                
                samplewise_std_normalization=True).flow(X_train_resized, y_train_encoded)
#val_gen=ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input,
#                samplewise_center=True,                
#                samplewise_std_normalization=True).flow(xValid, yValid, shuffle=False)        
test_gen=ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input,
                samplewise_center=True,                
                samplewise_std_normalization=True).flow(X_test_resized, y_test_encoded, shuffle=False)

In [ ]:
Top=False
weights=None
layer_cut=-6
lr_rate=.001
rand_seed=128
epochs=25
mobile = tf.keras.applications.mobilenet.MobileNet( include_top=Top,
                                                           input_shape=(224,224,3),
                                                           pooling='avg', weights='imagenet',
                                                          )  
                 
x=mobile.layers[layer_cut].output
x = Flatten()(x)
x=Dense(128, kernel_regularizer = regularizers.l2(l = 0.015), activation='relu')(x)

x=Dropout(rate=.5, seed=rand_seed)(x)
predictions=Dense (2, activation='softmax')(x)
model = Model(inputs=mobile.input, outputs=predictions)
        
for layer in model.layers:
    layer.trainable=True
model.compile(Adam(lr=lr_rate), loss='categorical_crossentropy', metrics=['accuracy'])